In [1]:
import math
import copy
import random
import time

In [2]:
import networkx as nx
import pandas as pd

In [3]:
from joblib import Parallel, delayed

In [4]:
def unweighted_search_information(G, paths):
    path_len = len(paths[0])-1

    ICs = []
    for path in paths:
        k = list(nx.degree(G, nbunch=path)) # return degrees (k) of nodes in the shorest path: [(node, degree), ...]
        temp = 0.0
        for i in range(len(k)-1):
            if i == 0:
                temp = 1.0/k[i][1]
            else:
                temp = temp*(1.0/(k[i][1]-1))
        
        ICs.append(temp)

    S = -1*math.log2(sum(ICs))

    return (path_len, S)

In [5]:
def weighted_search_information(G, paths):
    path_len = len(paths[0])-1

    ICs = []
    for path in paths:
        temp = 0.0
        for i in range(len(path)-1):
            if i == 0:
                successor_weights = dict([(successor, G.nodes[successor]['pagerank_score']) for successor in G.neighbors(path[i])])
                total_weight = sum([v for k, v in successor_weights.items()])
                temp = successor_weights[path[i+1]]/total_weight
            else:
                successor_weights = {}
                for successor in G.neighbors(path[i]):
                    if successor == path[i-1]: # ignore in node
                        continue
                    else:
                        successor_weights[successor] = G.nodes[successor]['pagerank_score']
                
                total_weight = sum([v for k, v in successor_weights.items()])
                temp = temp*(successor_weights[path[i+1]]/total_weight)

        ICs.append(temp)

    S = -1*math.log2(sum(ICs))

    return (path_len, S)

In [6]:
def seach_information(G, source, target, weighted=False):
    try:
        paths = [p for p in nx.all_shortest_paths(G, source, target)]
    except nx.NetworkXNoPath:
        return (float('inf'), '.')
    
    
    if weighted:
        path_len, S = weighted_search_information(G, paths)
    else:
        path_len, S = unweighted_search_information(G, paths)
    return (path_len, S)

In [7]:
def check_each_path(G, source, target, weighted=False):
    paths = [p for p in nx.all_shortest_paths(G, source, target)]

    if weighted:
        path_info = []
        for p in paths:
            path_len, S = weighted_search_information(G, [p])
            path_info.append((S, p))

        path_len, S = weighted_search_information(G, paths)
        print('{}--->{}, Search information:{} bits'.format(source, target, S))

        path_info.sort()
        print('\nPath list:')
        for p in path_info:
            print()
            print('Search information:{0}'.format(p[0]))
            print('-->'.join(p[1]))

    else:
        path_info = []
        for p in paths:
            path_len, S = unweighted_search_information(G, [p])
            path_info.append((S, p))

        path_len, S = unweighted_search_information(G, paths)
        print('{}--->{}, Search information:{} bits (unweighted)'.format(source, target, S))

        path_info.sort()
        print('\nPath list:')
        for p in path_info:
            print()
            print('Search information:{0}'.format(p[0]))
            print('-->'.join(p[1]))
    

In [8]:
def network_with_randomized_signature(G, original_signature):
    g = copy.deepcopy(G)
    
    nodes = list(g.nodes())
    
    used_nodes = set(original_signature.keys()) & set(nodes)
    
    random_genes = random.sample(nodes, len(used_nodes))
    map_table = dict(zip(used_nodes, random_genes))
    
    pseudo_signature = {}
    for k,v in original_signature.items():
        if k in used_nodes:
            pseudo_signature[map_table[k]] = v
        else:
            pseudo_signature[k] = v
    
    random_pagerank_scores = nx.pagerank(g, personalization=pseudo_signature)
    
    for n in g.nodes():
        g.nodes[n]['pagerank_score'] = random_pagerank_scores[n]
        
    return g
    

1. Loading protein-protein interaction network (STRING)

In [9]:
PPI = nx.read_gml('STRING750.gml')

2. Loading gene expression programs extracted from fetal organ development scRNA-seq data

In [10]:
programs = pd.read_csv('VHL_kidney_GEPs_T.csv', index_col=0)

3. Selecting signature of tissue-specific program

In [11]:
name_of_interested_program = 'P9'
topN = 100

In [12]:
NMF_weights = programs.loc[:, name_of_interested_program]
signature = NMF_weights.sort_values(ascending=False)[:topN]
signature = dict(signature)

In [13]:
shared = set(signature.keys()) & set(PPI.nodes())
print('There are {} proteins in PPI'.format(PPI.number_of_nodes()))
print('There are {} signature proteins'.format(len(signature)))
print('{} signature proteins are included in PPI network'.format(len(shared)))

There are 15193 proteins in PPI
There are 100 signature proteins
100 signature proteins are included in PPI network


4. Conducting personalized PageRank algorithm to assign preferential scores for each node in pathways, 
which representing the influence of tissue-specific GEP on whole cell signaling

In [14]:
pagerank_scores = nx.pagerank(PPI, personalization=signature)

In [15]:
for n in PPI.nodes():
    PPI.nodes[n]['pagerank_score'] = pagerank_scores[n]

In [16]:
pagerank_scores_df = pd.DataFrame([(k,v) for k,v in pagerank_scores.items()], columns=['Protein', 'PageRank_Score'])
pagerank_scores_df = pagerank_scores_df.sort_values(by='PageRank_Score', ascending=False)
pagerank_scores_df.head()
# pagerank_scores_df.to_csv('pagerank_scores.csv', index=False) # alternative

,Protein,PageRank_Score
2071,PLCB1,0.049114
5585,MAML2,0.017223
3803,HDAC9,0.010683
12565,LPHN3,0.010060
398,DOCK4,0.005710


5. Search information analysis

In [17]:
source = 'VHL'
target_proteins = [line.rstrip() for line in open('TF_names_v_1.01.txt') ] # downloaded from http://humantfs.ccbr.utoronto.ca/download.php

number_of_target_proteins = len(target_proteins)

validated_targets = set(PPI.nodes()) & set(target_proteins)
validated_targets = validated_targets - set(source)

validated_targets  = list(validated_targets)

number_of_target_protein_in_PPI = len(validated_targets)

print('There are {0} targets in input file, and {1} of them can be found in PPI'.format(number_of_target_proteins, number_of_target_protein_in_PPI))

There are 1639 targets in input file, and 1040 of them can be found in PPI


In [18]:
def random_generator(i, PPI, signature, validated_targets):
    randomized_pathways = network_with_randomized_signature(PPI, signature)

    random_delta_S = []
    for t in validated_targets:
        _, randomized_unweighted_info = seach_information(randomized_pathways, source, t, weighted=False)
        _, randomized_weighted_info = seach_information(randomized_pathways, source, t, weighted=True)
    
        r_delta_S = randomized_weighted_info - randomized_unweighted_info   
        
        random_delta_S.append(r_delta_S)
    
    return(random_delta_S)

In [ ]:
#------randomization-----
#------random seeds-------
N = 1000
start_time = time.time()
all_random_delta_S = Parallel(n_jobs=10)(delayed(random_generator)(i, PPI, signature, validated_targets) for i in range(N))
end_time = time.time()

In [ ]:
print('Running time on randomization: {} h, N={}'.format((end_time-start_time)/3600, N))

In [ ]:
random_info_change_matrix = pd.DataFrame(all_random_delta_S, columns=validated_targets)
random_info_change_matrix.head()

In [ ]:
obs_result = [] # observed results

for t in validated_targets:
    path_len, unweighted_info = seach_information(PPI, source, t, weighted=False)
    _, weighted_info = seach_information(PPI, source, t, weighted=True)
        
    delta_S = weighted_info-unweighted_info
                 
    if delta_S >= 0:
        empirical_p_value = sum(random_info_change_matrix[t] >= delta_S)/float(N)
    else:
        empirical_p_value = sum(random_info_change_matrix[t] <= delta_S)/float(N)
              
    obs_result.append([source, t, path_len, weighted_info, unweighted_info, delta_S, empirical_p_value])
        
obs_info_change_df = pd.DataFrame(obs_result, columns=['Source', 'Target', 'Path_Length', 'w_S', 'unw_S', 'Delta_S', 'Empirical_P_Value'])

In [ ]:
obs_info_change_df = obs_info_change_df.sort_values(by='Empirical_P_Value', ascending=True)
obs_info_change_df.to_csv('observed_info_change.csv', index=False) # alternative
random_info_change_matrix.to_csv('random_info_change.csv', index=False) # alternative

6. Annotating genes with their role in cancer

In [29]:
info_change_df = pd.read_csv('observed_info_change.csv')

In [ ]:
','.join(info_change_df[info_change_df['Empirical_P_Value'] < 0.05]['Target']) # them upload genes to NCG website: http://www.network-cancer-genes.org/index.php

7. Investigating paths

In [18]:
source = 'VHL'
target = 'PAX8'

In [19]:
check_each_path(PPI, source, target, weighted=False)

VHL--->PAX8, Search information:13.324650016220602 bits (unweighted)

Path list:

Search information:15.852090548114512
VHL-->EGLN2-->PRKCG-->PAX8

Search information:16.452820364693984
VHL-->EGLN2-->PRKCB-->PAX8

Search information:16.61108244861072
VHL-->EGLN3-->PRKCG-->PAX8

Search information:16.795507019748147
VHL-->EGLN1-->PRKCG-->PAX8

Search information:16.944208750102945
VHL-->EGLN2-->PRKCA-->PAX8

Search information:17.21181226519019
VHL-->EGLN3-->PRKCB-->PAX8

Search information:17.396236836327617
VHL-->EGLN1-->PRKCB-->PAX8

Search information:17.70320065059915
VHL-->EGLN3-->PRKCA-->PAX8

Search information:17.887625221736577
VHL-->EGLN1-->PRKCA-->PAX8

Search information:18.981199337048775
VHL-->ARNT-->SMAD3-->PAX8

Search information:19.133202430493824
VHL-->PIAS4-->SMAD3-->PAX8

Search information:19.546796512903
VHL-->PRKCD-->PRKCB-->PAX8

Search information:19.704805381366533
VHL-->TP53-->WT1-->PAX8

Search information:19.840304633736803
VHL-->HIF1A-->PPARG-->PAX8

Sear

In [20]:
check_each_path(PPI, source, target, weighted=True)

VHL--->PAX8, Search information:13.472396218266157 bits

Path list:

Search information:14.337083613357605
VHL-->TP53-->WT1-->PAX8

Search information:18.298035809667873
VHL-->ARNT-->SMAD3-->PAX8

Search information:18.426058015846312
VHL-->FN1-->PRKACA-->PAX8

Search information:18.749829497664816
VHL-->FN1-->PRKACB-->PAX8

Search information:19.003346411451396
VHL-->FN1-->SMAD3-->PAX8

Search information:19.528605392358624
VHL-->PRKCD-->PRKCB-->PAX8

Search information:19.601503585484977
VHL-->EP300-->PRKACB-->PAX8

Search information:19.625123192093568
VHL-->PRKCD-->PRKCA-->PAX8

Search information:19.854618196518757
VHL-->EP300-->SMAD3-->PAX8

Search information:19.85807669936516
VHL-->EP300-->RXRA-->PAX8

Search information:19.883865522122235
VHL-->EGLN2-->PRKCG-->PAX8

Search information:19.891621096579616
VHL-->EGLN2-->PRKCB-->PAX8

Search information:19.98477412907061
VHL-->EGLN2-->PRKCA-->PAX8

Search information:20.027864136190892
VHL-->TP53-->SMAD3-->PAX8

Search information